## C/C++ integration with Covalent

### Interfacing with C

We use a simple example of approximating the value of $\pi$ by computing the area under one quadrant of a unit circle. To do this, we use the `Reimann` sum approach where we break the region down into many rectangles of width $dx$ and height given by the equation of the circle

$$y = \sqrt{1 - x^2}, x \in [0, 1]$$

We write a C python module, compile it into a shared library and import it in our workflow. Doing this, as far as Covalent is concerned it is dealing with an ordinary Python module with routine Python functions accepting the necessary arguments.

The module source code can be found in `cpiapprox/main.c` and the method of interest is `compute_pi`. The module follows the Python/C API standard and defines a module named `cpiapprox` with a single method `compute_pi` that accepts an `unsigned int` as an argument a.k.a the number of partitions to divide the area in. Greater the number of partition, better the approximation of $\pi$.


### Compilation

To compile the source code into a shared library that can be imported simple run

```bash
python setup.py build
python setup.py install
```
from the directory containing the source code.


In [ ]:
import cpiapprox
import covalent as ct
from typing import List

In [ ]:
@ct.electron
def compute_pi(N: int):
    """
    :param N: number of partitions to use to approximate PI
    """
    return cpiapprox.compute_pi(N)

@ct.lattice
def approximate_pi(partitions: List[int]):
    results = []
    for n in partitions:
        results.append(compute_pi(n))
    return results


dispatch_id = ct.dispatch(approximate_pi)([10, 10000, 1000000, 1000000000])
result = ct.get_result(dispatch_id, wait=True)
print(result)


### Interfacing with C++

Covalent can be interfaced with C++ in a similar manner if one can expose their `C++` library as a Python module to Covalent. There are numerous open source wrappers available like `PyBind11/SWIG`. We here show how pure C++ functions can be invoked as Covalent electrons after being exposed via `PyBind11`.

More information on `PyBind11` can be found [here](https://github.com/pybind/pybind11). To install `pybind11`, simply run

```bash
pip install pybind11
```

As an example, we write a `C++` module `cpparithmetic` that performs simple elementwise vector arthimetic on two input `std::vectors`. The source code for this module can be found in `cpparthimetic/main.cc`.

To compile the source, use the provided `setup.py` file.

```bash
python setup.py build
python setup.py install
```

In [1]:
import cpparthimetic
import covalent as ct
import random
from typing import List
import time

### C++ workflows

In [15]:
@ct.electron
def generate_lists(size: int):
    return [random.uniform(1, 10) for i in range(size)]

@ct.electron
def cppadd(a, b):
    return cpparthimetic.vecadd(a, b)

@ct.electron
def cppmul(a, b):
    return cpparthimetic.vecmul(a, b)

@ct.electron
def cppdiv(a, b):
    return cpparthimetic.vecdiv(a, b)

@ct.lattice
def cppadd_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppadd(a, b)
    return c

@ct.lattice
def cppmul_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppmul(a, b)
    return c

@ct.lattice
def cppdiv_workflow(size):
    a = generate_lists(size)
    b = generate_lists(size)
    c = cppdiv(a, b)
    return c

In [16]:
N = 2**4

In [17]:
dispatch_ids = [ct.dispatch(cppadd_workflow)(N), ct.dispatch(cppmul_workflow)(N), ct.dispatch(cppdiv_workflow)(N)]

In [19]:
results = [ct.get_result(dispatch_id, wait=True).result for dispatch_id in dispatch_ids]

In [20]:
print(results)

[[8.262604904861552, 5.7566025790075575, 11.225241943046143, 7.297648911278621, 6.2004686100115975, 10.51496835221824, 13.714510093306387, 13.611023313699421, 12.144342316776443, 8.83126056429222, 7.38777641805463, 14.92542194880808, 9.001418834171856, 4.615744987911637, 8.84215084864721, 16.820841311326216], [51.773142933143276, 32.641198291348374, 63.259810113064944, 46.422185568189605, 16.832517090292864, 10.165527651589564, 24.91730312926613, 12.850439327862977, 4.72067092473866, 23.250522299938282, 40.86972251805796, 41.89140006732969, 32.74110262789796, 5.2884212352824935, 54.73076474361397, 14.165467704363962], [0.38921842387660177, 2.0585561806582144, 1.0886773829577225, 4.3297955353924475, 1.1896746314004352, 0.9525462232698703, 0.39332332223009064, 1.1590212034679328, 2.0449599797709386, 0.3091788475869194, 0.3393931532044408, 0.8876535296309052, 0.1706004840790428, 2.0798134895030134, 0.7698803193989625, 1.0122915751941892]]
